In [1]:
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'


# Obtain run information

The following example shows how to obtain information from a finished
Auto-sklearn run. In particular, it shows:
* how to query which models were evaluated by Auto-sklearn
* how to query the models in the final ensemble
* how to get general statistics on the what Auto-sklearn evaluated

Auto-sklearn is a wrapper on top of
the sklearn models. This example illustrates how to interact
with the sklearn components directly, in this case a PCA preprocessor.


In [2]:
import sklearn.datasets
import sklearn.metrics

import autosklearn.classification

## Data Loading



In [3]:
X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, random_state=1)

## Build and fit the classifier



In [4]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=30,
    per_run_time_limit=10,
    disable_evaluator_output=False,
    # To simplify querying the models in the final ensemble, we
    # restrict auto-sklearn to use only pca as a preprocessor
    include_preprocessors=['pca'],
)
automl.fit(X_train, y_train, dataset_name='breast_cancer')

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


AutoSklearnClassifier(dask_client=None,
                      delete_output_folder_after_terminate=True,
                      delete_tmp_folder_after_terminate=True,
                      disable_evaluator_output=False,
                      ensemble_memory_limit=1024, ensemble_nbest=50,
                      ensemble_size=50, exclude_estimators=None,
                      exclude_preprocessors=None, get_smac_object_callback=None,
                      include_estimators=None, include_preprocessors=['pca'],
                      initial_configurations_via_metalearning=25,
                      logging_config=None, max_models_on_disc=50,
                      metadata_directory=None, metric=None,
                      ml_memory_limit=3072, n_jobs=None, output_folder=None,
                      per_run_time_limit=10, resampling_strategy='holdout',
                      resampling_strategy_arguments=None, seed=1,
                      smac_scenario_args=None, time_left_for_this_task=30,


## Predict using the model



In [5]:
predictions = automl.predict(X_test)
print("Accuracy score:{}".format(
    sklearn.metrics.accuracy_score(y_test, predictions))
)

Accuracy score:0.9440559440559441


## Report the models found by Auto-Sklearn

Auto-sklearn uses
`Ensemble Selection <https://www.cs.cornell.edu/~alexn/papers/shotgun.icml04.revised.rev2.pdf>`_
to construct ensembles in a post-hoc fashion. The ensemble is a linear
weighting of all models constructed during the hyperparameter optimization.
This prints the final ensemble. It is a list of tuples, each tuple being
the model weight in the ensemble and the model itself.



In [6]:
print(automl.show_models())

[(0.360000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'passive_aggressive', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'median', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer', 'feature_preprocessor:__choice__': 'pca', 'classifier:passive_aggressive:C': 0.00029343005629408535, 'classifier:passive_aggressive:average': 'False', 'classifier:passive_aggressive:fit_intercept': 'True', 'classifier:passive_aggressive:loss': 'squared_hinge', 'classifier:passive_aggressive:tol': 0.0006217675098852786, 'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': 0.08644440750922357, 'data_preprocessing:numerical_transformer:rescaling:quantile_transformer:n_

## Report statistics about the search

Print statistics about the auto-sklearn run such as number of
iterations, number of models failed with a time out etc.



In [7]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: breast_cancer
  Metric: accuracy
  Best validation score: 0.971631
  Number of target algorithm runs: 10
  Number of successful target algorithm runs: 10
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0



## Detailed statistics about the search - part 1

Auto-sklearn also keeps detailed statistics of the hyperparameter
optimization procedurce, which are stored in a so-called
`run history <https://automl.github.io/SMAC3/master/apidoc/smac.
runhistory.runhistory.html#smac.runhistory# .runhistory.RunHistory>`_.



In [8]:
print(automl.automl_.runhistory_)

Runs are stored inside an ``OrderedDict`` called ``data``:



In [9]:
print(len(automl.automl_.runhistory_.data))

11


Let's iterative over all entries



In [10]:
for run_key in automl.automl_.runhistory_.data:
    print('#########')
    print(run_key)
    print(automl.automl_.runhistory_.data[run_key])

#########
RunKey(config_id=1, instance_id='{"task_id": "breast_cancer"}', seed=0, budget=0.0)
RunValue(cost=0.08510638297872342, time=1.98146390914917, status=<StatusType.SUCCESS: 1>, starttime=1603986204.0138662, endtime=1603986206.0106125, additional_info={'duration': 1.9145457744598389, 'num_run': 2, 'train_loss': 0.0, 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=2, instance_id='{"task_id": "breast_cancer"}', seed=0, budget=0.0)
RunValue(cost=0.1063829787234043, time=1.580573558807373, status=<StatusType.SUCCESS: 1>, starttime=1603986206.0159862, endtime=1603986207.6095388, additional_info={'duration': 1.5203707218170166, 'num_run': 3, 'train_loss': 0.14385964912280702, 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=3, instance_id='{"task_id": "breast_cancer"}', seed=0, budget=0.0)
RunValue(cost=0.04255319148936165, time=0.4011805057525635, status=<StatusType.SUCCESS: 1>, starttime=1603986207.6136591, endtime=1603986208.029248, additio

and have a detailed look at one entry:



In [11]:
run_key = list(automl.automl_.runhistory_.data.keys())[0]
run_value = automl.automl_.runhistory_.data[run_key]

The ``run_key`` contains all information describing a run:



In [12]:
print("Configuration ID:", run_key.config_id)
print("Instance:", run_key.instance_id)
print("Seed:", run_key.seed)
print("Budget:", run_key.budget)

Configuration ID: 1
Instance: {"task_id": "breast_cancer"}
Seed: 0
Budget: 0.0


and the configuration can be looked up in the run history as well:



In [13]:
print(automl.automl_.runhistory_.ids_config[run_key.config_id])

Configuration:
  balancing:strategy, Value: 'none'
  classifier:__choice__, Value: 'random_forest'
  classifier:random_forest:bootstrap, Value: 'True'
  classifier:random_forest:criterion, Value: 'gini'
  classifier:random_forest:max_depth, Constant: 'None'
  classifier:random_forest:max_features, Value: 0.5
  classifier:random_forest:max_leaf_nodes, Constant: 'None'
  classifier:random_forest:min_impurity_decrease, Constant: 0.0
  classifier:random_forest:min_samples_leaf, Value: 1
  classifier:random_forest:min_samples_split, Value: 2
  classifier:random_forest:min_weight_fraction_leaf, Constant: 0.0
  data_preprocessing:categorical_transformer:categorical_encoding:__choice__, Value: 'one_hot_encoding'
  data_preprocessing:categorical_transformer:category_coalescence:__choice__, Value: 'minority_coalescer'
  data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction, Value: 0.01
  data_preprocessing:numerical_transformer:imputation:strategy, V

The only other important entry is the budget in case you are using
auto-sklearn with
`successive halving <../60_search/example_successive_halving.html>`_.
The remaining parts of the key can be ignored for auto-sklearn and are
only there because the underlying optimizer, SMAC, can handle more general
problems, too.



The ``run_value`` contains all output from running the configuration:



In [14]:
print("Cost:", run_value.cost)
print("Time:", run_value.time)
print("Status:", run_value.status)
print("Additional information:", run_value.additional_info)
print("Start time:", run_value.starttime)
print("End time", run_value.endtime)

Cost: 0.08510638297872342
Time: 1.98146390914917
Status: StatusType.SUCCESS
Additional information: {'duration': 1.9145457744598389, 'num_run': 2, 'train_loss': 0.0, 'configuration_origin': 'Initial design'}
Start time: 1603986204.0138662
End time 1603986206.0106125


Cost is basically the same as a loss. In case the metric to optimize for
should be maximized, it is internally transformed into a minimization
metric. Additionally, the status type gives information on whether the run
was successful, while the additional information's most interesting entry
is the internal training loss. Furthermore, there is detailed information
on the runtime available.



As an example, let's find the best configuration evaluated. As
Auto-sklearn solves a minimization problem internally, we need to look
for the entry with the lowest loss:



In [15]:
losses_and_configurations = [
    (run_value.cost, run_key.config_id)
    for run_key, run_value in automl.automl_.runhistory_.data.items()
]
losses_and_configurations.sort()
print("Lowest loss:", losses_and_configurations[0][0])
print(
    "Best configuration:",
    automl.automl_.runhistory_.ids_config[losses_and_configurations[0][1]]
)

Lowest loss: 0.028368794326241176
Best configuration: Configuration:
  balancing:strategy, Value: 'none'
  classifier:__choice__, Value: 'passive_aggressive'
  classifier:passive_aggressive:C, Value: 0.00029343005629408535
  classifier:passive_aggressive:average, Value: 'False'
  classifier:passive_aggressive:fit_intercept, Constant: 'True'
  classifier:passive_aggressive:loss, Value: 'squared_hinge'
  classifier:passive_aggressive:tol, Value: 0.0006217675098852786
  data_preprocessing:categorical_transformer:categorical_encoding:__choice__, Value: 'one_hot_encoding'
  data_preprocessing:categorical_transformer:category_coalescence:__choice__, Value: 'minority_coalescer'
  data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction, Value: 0.08644440750922357
  data_preprocessing:numerical_transformer:imputation:strategy, Value: 'median'
  data_preprocessing:numerical_transformer:rescaling:__choice__, Value: 'quantile_transformer'
  data_preproce

## Detailed statistics about the search - part 2

To maintain compatibility with scikit-learn, Auto-sklearn gives the
same data as
`cv_results_ <https://scikit-learn.org/stable/modules/generated/sklearn.
model_selection.GridSearchCV.html>`_.



In [16]:
print(automl.cv_results_)

{'mean_test_score': array([0.91489362, 0.89361702, 0.95744681, 0.94326241, 0.94326241,
       0.89361702, 0.85815603, 0.97163121, 0.89361702, 0.94326241]), 'mean_fit_time': array([1.98146391, 1.58057356, 0.40118051, 0.40705061, 0.47366381,
       0.34416032, 0.58366561, 0.4089849 , 0.47272778, 1.81905317]), 'params': [{'balancing:strategy': 'none', 'classifier:__choice__': 'random_forest', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'standardize', 'feature_preprocessor:__choice__': 'pca', 'classifier:random_forest:bootstrap': 'True', 'classifier:random_forest:criterion': 'gini', 'classifier:random_forest:max_depth': 'None', 'classifier:random_forest:max_features': 0.5, 'classifier:random_forest:max_l

## Inspect the components of the best model

Iterate over the components of the model and print
The explained variance ratio per stage



In [17]:
for i, (weight, pipeline) in enumerate(automl.get_models_with_weights()):
    for stage_name, component in pipeline.named_steps.items():
        if 'preprocessor' in stage_name:
            print(
                "The {}th pipeline has a explained variance of {}".format(
                    i,
                    # The component is an instance of AutoSklearnChoice.
                    # Access the sklearn object via the choice attribute
                    # We want the explained variance attributed of
                    # each principal component
                    component.choice.preprocessor.explained_variance_ratio_
                )
            )

The 0th pipeline has a explained variance of [0.45954161 0.18012095 0.09808836 0.06332952 0.05871665 0.03690454]
The 1th pipeline has a explained variance of [0.43295688 0.1790573 ]
The 2th pipeline has a explained variance of [0.49503611 0.16649281 0.09111888 0.07213284 0.04865917]
The 3th pipeline has a explained variance of [0.98080571]
The 4th pipeline has a explained variance of [0.73153912 0.10584539 0.07328331]
The 5th pipeline has a explained variance of [0.43295688 0.1790573  0.11173757]
The 6th pipeline has a explained variance of [0.43295688 0.1790573  0.11173757 0.06807243 0.05946115 0.03706299
 0.0238431  0.01493261 0.01376414]
The 7th pipeline has a explained variance of [4.32956881e-01 1.79057296e-01 1.11737571e-01 6.80724345e-02
 5.94611519e-02 3.70629898e-02 2.38430977e-02 1.49326086e-02
 1.37641366e-02 1.13704890e-02 1.03737258e-02 8.74116751e-03
 7.57629717e-03 4.86528503e-03 3.32225143e-03 2.55773043e-03
 2.20759805e-03 1.88675402e-03 1.36245140e-03 1.03409213e-03
 